# 하나은행 환율 DB 저장
* 30년치 저장하기
  1) 1페이지 자료 추출해서 데이터프레임으로 출력하기
  2) 과거 환율자료 1995.01.03부터 2024.07.31까지 (주5일 가능?)
  3) DB 가져올 때마다 저장하기
----------------------------------------------------------
* 오늘부터 외환시장 장 마감 후 데이터 수집 자동화
  1) 1페이지 자료 추출해서 데이터프레임으로 출력하기
  2) DB에 가져올 때마다 저장하기
  3) 실행파일로 만들기
  4) 스케줄러에 등록
----------------------------------------------------------
* 정치, 사회 뉴스 Naver api 로 수집

In [6]:
#  1) 1페이지 자료 추출해서 데이터프레임으로 출력하기

In [7]:
import selenium
print(selenium.__version__)

4.31.1


In [8]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager 
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup as bs
import time
from datetime import date

# 크롬 옵션즈에 정보를 담아 사람인 것 처럼 만들기
options = webdriver.ChromeOptions()
options.add_experimental_option("excludeSwitches", ['enable-logging'])
# options.add_argument("Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/127.0.0.0 Safari/537.36")
# options.add_argument('lang=ko_KR')
options.add_argument("disable-gpu")
options.add_argument("--headless")
options.add_argument("--no-sandbox")
options.add_argument("--disable-dev-shm-usage")

# 크롬 웹브라우저 드라이버 자동 다운로드
driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()))
driver.set_window_size(1920, 1080) #웹브라우저 해상도 조절
driver.get("https://www.kebhana.com/cms/rate/index.do?contentUrl=/cms/rate/wpfxd651_01i.do#//HanaBank")

In [10]:
#tmpInqStrDt
#inqFrm > table > tbody > tr:nth-child(2) > td > div
# type="text"
# //*[@id="tmpInqStrDt"]

calender = driver.find_element(By.CSS_SELECTOR, "#tmpInqStrDt")
calender.clear()
calender.send_keys("2024-07-31") #+Keys.ENTER)
# serch_box.send_keys("핀테크"+Keys.ENTER) 핀테크 검색됨

In [11]:
#HANA_CONTENTS_DIV > div.btnBoxCenter > a
search_button = driver.find_element(By.CSS_SELECTOR, "#HANA_CONTENTS_DIV > div.btnBoxCenter > a") #조회 버튼
search_button.click() #검색

In [12]:
import pandas as pd

In [13]:
page_html = driver.page_source
df = pd.read_html(page_html)[1]
print(df.columns)
#날짜 컬럼을 추가


MultiIndex([(       '통화',        '통화',        '통화'),
            (       '현찰',      '사실 때',        '환율'),
            (       '현찰',      '사실 때',    'Spread'),
            (       '현찰',      '파실 때',        '환율'),
            (       '현찰',      '파실 때',    'Spread'),
            (       '송금',      '보낼 때',      '보낼 때'),
            (       '송금',      '받을 때',      '받을 때'),
            (  'T/C 사실때',   'T/C 사실때',   'T/C 사실때'),
            ('외화 수표 파실때', '외화 수표 파실때', '외화 수표 파실때'),
            (   '매매 기준율',    '매매 기준율',    '매매 기준율'),
            (    '환가 료율',     '환가 료율',     '환가 료율'),
            (   '미화 환산율',    '미화 환산율',    '미화 환산율')],
           )


C:\Users\5-12\AppData\Local\Temp\ipykernel_19108\4288707641.py:2: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(page_html)[1]


In [ ]:
#  2) 과거 환율자료 1995.01.03부터 2024.07.31까지 (주5일 가능?)

In [14]:
import time
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [15]:
period = pd.date_range(start="2024-07-30", end="2024-07-31")
date_list = period.tolist()
print(date_list)

[Timestamp('2024-07-30 00:00:00'), Timestamp('2024-07-31 00:00:00')]


In [16]:
date_list_str = [date.strftime('%Y%m%d') for date in period]
print(date_list_str)

['20240730', '20240731']


In [17]:
cols =("통화","현찰_살때_환율","현찰_살때_spread","현찰_팔때_환율","현찰_팔때_spread","송금_보낼때","송금_받을때",
       "T/C_살때", "외화수표_팔때", "매매_기준율", "환가_료율","미화_환산율", "date")
sorted_cols = ( "date", "통화","현찰_살때_환율","현찰_살때_spread","현찰_팔때_환율","현찰_팔때_spread","송금_보낼때","송금_받을때",
       "T/C_살때", "외화수표_팔때", "매매_기준율", "환가_료율","미화_환산율")

In [18]:
df = pd.DataFrame()
today = str(date.today())

for date_str in date_list_str:
    calender = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "#tmpInqStrDt")))
    calender.clear()
    calender.send_keys(date_str)

    # 로딩 레이어가 사라질 때까지 대기
    WebDriverWait(driver, 10).until(EC.invisibility_of_element_located((By.ID, "OPB_loadingLayerID_generatedByJSOPB_modalMaskID_generatedByJS")))

    # 검색 버튼 클릭 시도
    try:
        search_button = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "#HANA_CONTENTS_DIV > div.btnBoxCenter > a")))
        search_button.click()
        
    except ElementClickInterceptedException:
        # JavaScript로 클릭 시도
        driver.execute_script("arguments[0].click();", search_button)

    time.sleep(2)  # 조회 결과가 로드될 시간을 기다림
    page_html = driver.page_source
    data = pd.read_html(page_html)[1]
    df = pd.concat([df, data], ignore_index = True)

df['date'] = today
cols =("통화","현찰_살때_환율","현찰_살때_spread","현찰_팔때_환율","현찰_팔때_spread","송금_보낼때","송금_받을때",
   "T/C_살때", "외화수표_팔때", "매매_기준율", "환가_료율","미화_환산율", "date")
sorted_cols = ( "date", "통화","현찰_살때_환율","현찰_살때_spread","현찰_팔때_환율","현찰_팔때_spread","송금_보낼때","송금_받을때",
   "T/C_살때", "외화수표_팔때", "매매_기준율", "환가_료율","미화_환산율")
df.columns = cols

all_data = df[[*sorted_cols]]#언패킹하여 넣기
display(all_data)
#all_data.columns = sorted_cols 시에는 컬럼만 바뀌고 데이터는 그대로유지됨

C:\Users\5-12\AppData\Local\Temp\ipykernel_19108\1018225450.py:23: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  data = pd.read_html(page_html)[1]
C:\Users\5-12\AppData\Local\Temp\ipykernel_19108\1018225450.py:23: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  data = pd.read_html(page_html)[1]


,date,통화,현찰_살때_환율,현찰_살때_spread,현찰_팔때_환율,현찰_팔때_spread,송금_보낼때,송금_받을때,T/C_살때,외화수표_팔때,매매_기준율,환가_료율,미화_환산율
0,2024-08-01,미국 USD,1409.23,1.75,1360.77,1.75,1398.50,1371.50,0.0,1368.74,1385.00,7.19371,1.0000
1,2024-08-01,일본 JPY (100),921.73,1.75,890.03,1.75,914.75,897.01,0.0,896.52,905.88,2.19682,0.6541
2,2024-08-01,유로 EUR,1527.47,1.99,1467.87,1.99,1512.64,1482.70,0.0,1480.38,1497.67,5.59600,1.0814
3,2024-08-01,중국 CNY,200.76,5.00,181.64,5.00,193.11,189.29,0.0,0.00,191.20,4.87455,0.1381
4,2024-08-01,홍콩 HKD,180.76,1.97,173.78,1.97,179.04,175.50,0.0,175.22,177.27,6.57333,0.1280
...,...,...,...,...,...,...,...,...,...,...,...,...,...
111,2024-08-01,리비아 LYD,0.00,0.00,0.00,0.00,286.38,280.72,0.0,0.00,283.55,0.00000,0.2068
112,2024-08-01,루마니아 RON,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.00,298.30,0.00000,0.2176
113,2024-08-01,미얀마 MMK,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.00,0.65,0.00000,0.0005
114,2024-08-01,에티오피아 ETB,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.00,17.19,0.00000,0.0125


In [ ]:
#  3) DB 가져올 때마다 저장하기 (korea_excahge_rates)

In [1]:
import os
import time
from sqlalchemy import create_engine, text
import pymysql
from dbenv2 import db, dbtype, id, pw, host, database

#pymysql을 MySQLdb로 설치
pymysql.install_as_MySQLdb()


#데이터베이스 엔진 생성
engine = create_engine("%s+%s://%s:%s@%s" % (db, dbtype, id, pw, host))
conn = engine.connect()

#데이터베이스가 없으면 생성
conn.execute(text('CREATE DATABASE IF NOT EXISTS korea_exchange_rate'))
conn.close()

#DB에 접속해서 저장하는 함수
def exchange_rate(df):
    from sqlalchemy import create_engine
    import pymysql
    from dbenv2 import db, dbtype, id, pw, host, database
    from datetime import date
    pymysql.install_as_MySQLdb()

    today = str(date.today()).replace("-","")

    engine = create_engine("%s+%s://%s:%s@%s/%s" % (db, dbtype, id, pw, host, database))
    conn = engine.connect()
    df.to_sql(f"{today[:6]}korea_exchange_rate", con=conn, if_exists='append', index=False)
    conn.close()

    return print(f"{'저장완료':<30s}", end="\r")

In [20]:
exchange_rate(all_data)

In [ ]:
date = '2024-08-01'

In [ ]:
all_data['date'] = date

In [ ]:
df #date 추가

,통화,현찰_살때_환율,현찰_살때_spread,현찰_팔때_환율,현찰_팔때_spread,송금_보낼때,송금_받을때,T/C_살때,외화수표_팔때,매매_기준율,환가_료율,미화_환산율,date
0,미국 USD,1394.99,1.75,1347.01,1.75,1384.40,1357.60,0.0,1354.87,1371.00,7.19233,1.0000,2024-08-01
1,일본 JPY (100),930.30,1.75,898.30,1.75,923.26,905.34,0.0,904.85,914.30,2.19682,0.6669,2024-08-01
2,유로 EUR,1513.99,1.99,1454.91,1.99,1499.29,1469.61,0.0,1467.32,1484.45,5.57100,1.0827,2024-08-01
3,중국 CNY,199.27,5.00,180.31,5.00,191.68,187.90,0.0,0.00,189.79,4.64409,0.1384,2024-08-01
4,홍콩 HKD,178.94,1.97,172.04,1.97,177.24,173.74,0.0,173.46,175.49,6.57366,0.1280,2024-08-01
5,태국 THB,40.51,5.00,36.28,6.00,38.97,38.21,0.0,38.17,38.59,4.52500,0.0281,2024-08-01
6,대만 TWD,47.41,13.10,37.73,10.00,0.00,0.00,0.0,0.00,41.92,3.53233,0.0306,2024-08-01
7,필리핀 PHP,25.86,10.00,21.59,8.20,23.74,23.28,0.0,0.00,23.51,8.34500,0.0171,2024-08-01
8,싱가포르 SGD,1046.53,1.99,1005.71,1.99,1036.38,1015.86,0.0,1014.00,1026.12,7.36633,0.7484,2024-08-01
9,호주 AUD,914.72,1.97,879.38,1.97,906.02,888.08,0.0,886.48,897.05,6.52233,0.6543,2024-08-01


In [ ]:
all_data.columns

MultiIndex([(       '통화',        '통화',        '통화'),
            (       '현찰',      '사실 때',        '환율'),
            (       '현찰',      '사실 때',    'Spread'),
            (       '현찰',      '파실 때',        '환율'),
            (       '현찰',      '파실 때',    'Spread'),
            (       '송금',      '보낼 때',      '보낼 때'),
            (       '송금',      '받을 때',      '받을 때'),
            (  'T/C 사실때',   'T/C 사실때',   'T/C 사실때'),
            ('외화 수표 파실때', '외화 수표 파실때', '외화 수표 파실때'),
            (   '매매 기준율',    '매매 기준율',    '매매 기준율'),
            (    '환가 료율',     '환가 료율',     '환가 료율'),
            (   '미화 환산율',    '미화 환산율',    '미화 환산율'),
            (     'date',          '',          '')],
           )

In [ ]:
all_data.columns = cols

In [ ]:
all_data

,통화,현찰_살때_환율,현찰_살때_spread,현찰_팔때_환율,현찰_팔때_spread,송금_보낼때,송금_받을때,T/C_살때,외화수표_팔때,매매_기준율,환가_료율,미화_환산율,date
0,미국 USD,1409.23,1.75,1360.77,1.75,1398.50,1371.50,0.0,1368.74,1385.00,7.19371,1.0000,2024-08-01
1,일본 JPY (100),921.73,1.75,890.03,1.75,914.75,897.01,0.0,896.52,905.88,2.19682,0.6541,2024-08-01
2,유로 EUR,1527.47,1.99,1467.87,1.99,1512.64,1482.70,0.0,1480.38,1497.67,5.59600,1.0814,2024-08-01
3,중국 CNY,200.76,5.00,181.64,5.00,193.11,189.29,0.0,0.00,191.20,4.87455,0.1381,2024-08-01
4,홍콩 HKD,180.76,1.97,173.78,1.97,179.04,175.50,0.0,175.22,177.27,6.57333,0.1280,2024-08-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...
111,리비아 LYD,0.00,0.00,0.00,0.00,286.38,280.72,0.0,0.00,283.55,0.00000,0.2068,2024-08-01
112,루마니아 RON,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.00,298.30,0.00000,0.2176,2024-08-01
113,미얀마 MMK,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.00,0.65,0.00000,0.0005,2024-08-01
114,에티오피아 ETB,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.00,17.19,0.00000,0.0125,2024-08-01
